In [17]:
import pandas as pd

import pymongo

Use [govuk-mongodb-content](https://github.com/alphagov/govuk-mongodb-content) to setup local mongodb instance, using documentation [here](https://docs.google.com/document/d/1RhJwC79XLryOpr1ELWfG0E1eni4dGMompOjOZrDADd0/edit#heading=h.qkjm4ngtcm81)

In [2]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

content_store_db = mongo_client['content_store']
content_store_collection = content_store_db['content_items']

In [3]:
CONTENT_ID_PROJECTION = {"content_id": 1}

In [9]:
FILTER_HAS_MANUAL_RELATED_LINKS = {
    "$or": [
#             standard related links
            {"expanded_links.ordered_related_items": {"$exists": True}},
               
#             related_mainstream_content link, e.g. see /guidance/work-out-if-youll-pay-the-scottish-rate-of-income-tax
            {"expanded_links.related_mainstream_content": {"$exists": True}},
        
#             quick_links, e.g. see /government/organisations/hm-revenue-customs/contact/creative-industry-tax-reliefs
            {"details.quick_links": {"$exists": True}} 
        ]}

In [14]:
pages_with_manual_related_links_cursor = content_store_collection.find(
    FILTER_HAS_MANUAL_RELATED_LINKS,
    CONTENT_ID_PROJECTION)

In [15]:
pages_with_manual_related_links = list(pages_with_manual_related_links_cursor)

In [19]:
manual_related_links_df = pd.DataFrame(pages_with_manual_related_links)

In [22]:
manual_related_links_df = manual_related_links_df.rename(
    columns = {'_id': 'page_path'})

In [24]:
manual_related_links_df.to_csv(
    'manually_curated_related_links_pages.csv',
    index=False)